In [4]:
from src.agents import Volatile

In [5]:
N_MOLECULE = 100000
volatiles = []
for i in range(N_MOLECULE):
    volatiles.append(Volatile())

In [6]:
for molecule in volatiles:
    for j in range(5000):
        molecule.migrate(2.989e-26)